<a href="https://colab.research.google.com/github/zen4s/IK/blob/master/X_WIP_subhra_ghosh_volcanoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Your Full Name - Subhra Ghosh

Your Uplevel Email Address - b206423@gmail.com

**Name of the Problem Statement of Submission** -


# Mini ML Project 1 - Volcano ML (MLSU August 2023)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import glob
import csv

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
import os

def get_subdirectory_names(directory_path):
  """Returns an array of names of subdirectories in the given directory."""

  subdirectory_names = []
  for entry in os.listdir(directory_path):
    full_path = os.path.join(directory_path, entry)
    if os.path.isdir(full_path):
      subdirectory_names.append(entry)

  subdirectory_names.sort()  # Sort the list of names

  return subdirectory_names

In [ ]:
#Convert Nano Radiant to scientific_notation Convertor
def nrad_to_scientific(nardstr):

  # Extract the numeric part and unit
  match = re.match(r"([0-9\.]+)nrad", nardstr)
  if match:
    radians = float(match.group(1)) * 1e-9
    scientific_notation = f"{radians:.6e}"
    return(scientific_notation)
  else:
    raise ValueError(f"Invalid format for nanoradian string: {tilt_sensor_value}")

In [ ]:
#Convert Nano Radiant for Tilt Angle at Eruption into its scentific format for use in graphs (as it repsents data at time greater than zero)

def df_nrad_to_num(df_nard):
  # Access the tilt_sensor value and its index
  tilt_sensor_value = df_nard.loc[df_nard[0] == 'tilt_erupt', 1].iloc[0]
  tilt_sensor_index = df_nard.index[df_nard[0] == 'tilt_erupt'].tolist()[0]  # Get the first index

  # Update the value directly using at
  df_nard.at[tilt_sensor_index, 1] = nrad_to_scientific(tilt_sensor_value)

  return(df_nard)

In [ ]:
# create a dictionary of Sensor Parameters
dict_sensor_parm = {
    'v': 'Poissons Ratio',
    'Patm': 'Atmospheric Pressure',
    'g': 'Gravity',
    'r': 'Radial Distance Tilt Station',
    'G': 'Shear Modulus',
    'rho': 'Magma Density',
    'mu': 'Magma Viscosity',
    'rc': 'Conduit Radius',
    'M': 'Gas Mass',
    'sigma': 'Standard Deviation',
    'tilt_erupt': 'Tilt Angle at Eruption (nrad)',
}
parm_full_name = np.array(list(dict_sensor_parm .values()))
print(parm_full_name)


['Poissons Ratio' 'Atmospheric Pressure' 'Gravity'
 'Radial Distance Tilt Station' 'Shear Modulus' 'Magma Density'
 'Magma Viscosity' 'Conduit Radius' 'Gas Mass' 'Standard Deviation'
 'Tilt Angle at Eruption (nrad)']


In [ ]:
# Example usage:
directory_path = '/gdrive/MyDrive/IK/datasets/proj/volcano/' # Replace with the actual path
volcano_names = get_subdirectory_names(directory_path)
print(volcano_names)

['Volcano1', 'Volcano10', 'Volcano2', 'Volcano3', 'Volcano4', 'Volcano5', 'Volcano6', 'Volcano7', 'Volcano8', 'Volcano9']


In [ ]:
df1_timeline = pd.DataFrame({0: [1, 2, 3], 1: ['a', 'b', 'c']})

df2_timeline = pd.DataFrame(columns=['id'])
df2_timeline = pd.concat([df2_timeline , df1_timeline], ignore_index=True)

df2_timeline['id'] = 'XX'
df2_timeline

,id,0,1
0,XX,1.0,a
1,XX,2.0,b
2,XX,3.0,c


In [ ]:
import pandas as pd    # Import pandas for data manipulation
import glob            # Import glob for finding files
import csv             # Import csv for reading text files

# **Pre-allocate DataFrames for storing data**
dfs_timeline = pd.DataFrame()               # Combined timeline data from all files
dfs_timeline_seq_with_id = pd.DataFrame()   # Timeline data with sequential IDs
df_sensor_timeline = pd.DataFrame()         # Sensor metadata
min_time = 0                                # Variable to track the earliest time

for volcano_name in volcano_names:
  #Folder name / Volcano name
  observationloc = directory_path + volcano_name +'/'
  # **Find all .txt files in the specified directory**
  observation_files = glob.glob(observationloc + '/*.txt')
  observation_count = 1                       # Counter for observation files

  # **Process each observation file**
  for file in observation_files:
      # **Create unique column names for each observation**
      obs_col_name = f'{volcano_name}_obs_{observation_count}'    # Column name for observation values
      sensor_col_name = f's_{obs_col_name}'                       # Column name for sensor values
      time_col_name = f'time_{obs_col_name}'                      # Column name for time values

      # **Read the file, skipping empty lines**
      with open(file, 'r') as f:
          non_empty_lines = [line.strip() for line in f if line.strip()]  # Filter empty lines
          all_rows = list(csv.reader(non_empty_lines))

      # **Separate metadata and timeline data**
      df1 = pd.DataFrame(all_rows[:11])  # Sensor metadata (first 11 rows)
      df1_timeline = pd.DataFrame(all_rows[11:])  # Tilt sensor timeline (remaining rows)
      df1_timeline = df1_timeline.apply(pd.to_numeric, errors='coerce')  # Convert to numeric

      df2_timeline = pd.DataFrame(columns=['id'])
      # **Transpose timeline data for easier manipulation**
      df1_timeline = df1_timeline.T
      df2_timeline = pd.concat([df2_timeline , df1_timeline], ignore_index=True)
      df2_timeline['id'] = obs_col_name
      df2_timeline = df2_timeline.rename(columns={0: 'time', 1 : 'sensor_reading'})

      # **Sort timeline data in descending order (as it's negative) and reset index**
      df1_timeline = df1_timeline.sort_values(by=0, ascending=False).reset_index(drop=True)

      # **Get the earliest time and track the longest timeline**
      first_sensor_time = float(df1_timeline.iloc[-1, 0])
      if min_time > first_sensor_time:
          min_time = first_sensor_time
          longest_timeline_list = df1_timeline.iloc[:, 0].tolist()

      # **Rename columns for clarity**
      df1_timeline = df1_timeline.rename(columns={0: time_col_name, 1: sensor_col_name})

      # **Convert nrad to scientific notation (function not shown)**
      df1 = df_nrad_to_num(df1)  # Assuming this function is defined elsewhere

      # **Rename metadata columns**
      df1 = df1.rename(columns={0: 'parameters', 1: obs_col_name})

      # **Combine sensor metadata from all files into a single DataFrame**
      if observation_count == 1:
          df_sensor_meta = df1.copy()
      else:
          #df_sensor_meta = pd.concat([df_sensor_meta, df1], ignore_index=True)
          dict = df1.set_index('parameters').to_dict()[obs_col_name]
          df_sensor_meta[obs_col_name] = df_sensor_meta['parameters'].apply(lambda x: dict[x])

      # **Append timeline data to the combined DataFrame**
      dfs_timeline = pd.concat([dfs_timeline, df1_timeline], axis=1)

      observation_count += 1

      # **Remove sensor data with time = 0**
      mask = df2_timeline['time'] != 0
      df2_timeline = df2_timeline[mask]

      #Add Sequential ID to df timeline data
      dfs_timeline_seq_with_id = pd.concat([dfs_timeline_seq_with_id, df2_timeline], ignore_index=True)

#Insert Timedata from the timeline with longest time slice
dfs_timeline.insert(0, 'time', longest_timeline_list)

#Drop other timelines once max timeline is inserted
#for i in range(observation_count-1):
#  dfs_timeline.pop(f'time_obs_{i+1}')

# Print results
print("df_sensor_meta:\n",df_sensor_meta)

#fill all NaN with Zeros
dfs_timeline = dfs_timeline.fillna(0)

print("dfs_timeline:\n", dfs_timeline)
print("dfs_timeline_Seq:\n", dfs_timeline_seq_with_id)


df_sensor_meta:
     parameters Volcano9_obs_1 Volcano9_obs_2 Volcano9_obs_3 Volcano9_obs_4  \
0            v           0.25           0.25           0.25           0.25   
1         Patm         100000         100000         100000         100000   
2            g           9.81           9.81           9.81           9.81   
3            r            500            500            500            500   
4            G       10^11.44       10^11.44       10^11.44       10^11.44   
5          rho           2400           2400           2400           2400   
6           mu        10^4.55        10^4.55        10^4.55        10^4.55   
7           rc             15             15             15             15   
8            M         10^5.9         10^5.9         10^5.9         10^6.3   
9        sigma            0.1           0.25            0.5           0.25   
10  tilt_erupt   1.152900e-09   1.152900e-09   1.152900e-09   2.012500e-09   

   Volcano9_obs_5 Volcano9_obs_6 Volcano9_obs_

<ipython-input-9-481b4b9212f5>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfs_timeline.insert(0, 'time', longest_timeline_list)


In [ ]:
dfs_timeline_seq_with_id['id'].unique()

array(['Volcano1_obs_1', 'Volcano1_obs_2', 'Volcano1_obs_3',
       'Volcano1_obs_4', 'Volcano1_obs_5', 'Volcano1_obs_6',
       'Volcano1_obs_7', 'Volcano1_obs_8', 'Volcano1_obs_9',
       'Volcano1_obs_10', 'Volcano1_obs_11', 'Volcano1_obs_12',
       'Volcano10_obs_1', 'Volcano10_obs_2', 'Volcano10_obs_3',
       'Volcano10_obs_4', 'Volcano10_obs_5', 'Volcano10_obs_6',
       'Volcano10_obs_7', 'Volcano10_obs_8', 'Volcano10_obs_9',
       'Volcano10_obs_10', 'Volcano10_obs_11', 'Volcano10_obs_12',
       'Volcano10_obs_13', 'Volcano10_obs_14', 'Volcano10_obs_15',
       'Volcano10_obs_16', 'Volcano10_obs_17', 'Volcano10_obs_18',
       'Volcano10_obs_19', 'Volcano10_obs_20', 'Volcano10_obs_21',
       'Volcano10_obs_22', 'Volcano10_obs_23', 'Volcano10_obs_24',
       'Volcano10_obs_25', 'Volcano10_obs_26', 'Volcano10_obs_27',
       'Volcano10_obs_28', 'Volcano10_obs_29', 'Volcano10_obs_30',
       'Volcano2_obs_1', 'Volcano2_obs_2', 'Volcano2_obs_3',
       'Volcano2_obs_4', 'Volca

In [ ]:
df_sensor_meta.insert(0, 'description', df_sensor_meta['parameters'].map(dict_sensor_parm))

# Print the DataFrame with sendor metadata
print(df_sensor_meta)

                      description  parameters Volcano9_obs_1 Volcano9_obs_2  \
0                  Poissons Ratio           v           0.25           0.25   
1            Atmospheric Pressure        Patm         100000         100000   
2                         Gravity           g           9.81           9.81   
3    Radial Distance Tilt Station           r            500            500   
4                   Shear Modulus           G       10^11.44       10^11.44   
5                   Magma Density         rho           2400           2400   
6                 Magma Viscosity          mu        10^4.55        10^4.55   
7                  Conduit Radius          rc             15             15   
8                        Gas Mass           M         10^5.9         10^5.9   
9              Standard Deviation       sigma            0.1           0.25   
10  Tilt Angle at Eruption (nrad)  tilt_erupt   1.152900e-09   1.152900e-09   

   Volcano9_obs_3 Volcano9_obs_4 Volcano9_obs_5 Vol

In [ ]:
# Read the first 10 rows of the sensor metadata into a DataFrame
df_sensor = pd.read_csv(observationloc+'V1observation1.dat', nrows=11, header=None, index_col=False)

# Print the DataFrame with sendor metadata
print(df_sensor,"\n")

#Covert the nano radient to number and rename the columns
df_sensor = df_nrad_to_num(df_sensor).rename(columns={0: 'parameters', 1: 'obs'})
print(df_sensor)

FileNotFoundError: [Errno 2] No such file or directory: '/gdrive/MyDrive/IK/datasets/proj/volcano/Volcano9/V1observation1.dat'

**Poisson's Ratio (v):**
Relevant to the elasticity of the volcanic material.
Determines the ratio of lateral contraction to longitudinal extension.

**Atmospheric Pressure (Patm):**
Influences the pressure conditions in the atmosphere.
Atmospheric pressure can impact the behavior of volcanic gases and magma.

**Gravity (g):**
The gravitational force experienced at the surface of the volcano.
Gravity affects the movement and dynamics of magma within the volcanic conduit.

**Radial Distance Tilt Station (r):**
Represents the radial distance of a tilt station from the volcano.
Tilt stations measure ground tilt, providing insights into volcanic activity.

**Shear Modulus (G):**
Related to the material's resistance to shear deformation.
Affects the ability of volcanic material to deform under stress.

**Magma Density (rho):**
Represents the density of the magma.
Magma density influences its behavior and flow dynamics.

**Magma Viscosity (mu):**
Describes the resistance of magma to flow.
Viscosity affects the speed and style of volcanic eruptions.

**Conduit Radius (rc):**
Refers to the radius of the volcanic conduit through which magma ascends.
Conduit radius influences the flow rate of magma.

**Gas Mass (M):**
Represents the mass of gases emitted during volcanic activity.
Gas mass is crucial for understanding the composition and effects of volcanic gases.

**Standard Deviation (sigma):**
A measure of the variability or dispersion of parameters.
Standard deviation can introduce variability in the synthetic dataset.

**Tilt Angle at Eruption (tilt_erupt):
Represents the tilt angle of the ground at the eruption point.
Tilt angles provide information about the movement and pressure of magma.**

In [ ]:
# Add a column C2 to the DataFrame by matching keys from column C1 with inserting value pair from a dictionary to C2
#df_sensor['description'] = df_sensor['parameters'].apply(lambda x: dict_sensor_parm[x])
#df_sensor = df_sensor.assign(description=df_sensor['parameters'].map(dict_sensor_parm))
df_sensor.insert(0, 'description', df_sensor['parameters'].map(dict_sensor_parm))

print(df_sensor)

In [ ]:
# Read the main sensor data, skipping the first 13 rows (metadata of the sensor)
# and setting no header or index column so both rows are read as data
# ----------------------------------------------------------------
df = pd.read_csv(observationloc+'V1observation1.dat', skiprows=13, header=None, index_col=False)

# Display DataFrame as extracted from file before tranformation
print(df.head())


In [ ]:

# Transpose the DataFrame and rename the columns for clarity
df = df.T.rename(columns={0: 'time', 1: 'sensor_value'})

#Explore Data: show the Observation Data in various basic ways
print(df.head())  # View first few rows
print(df.info())  # Check data types
print(df.describe())  # Get summary statistics

In [ ]:
# Time 0 is erruption, this displays last t-15 seconds before erruption

df.tail(15)


In [ ]:
time_column = "time"  # Replace with the actual column name containing time data
value_column = "sensor_value"  # Replace with the actual column name containing values

time = df[time_column]
values = df[value_column]

plt.figure(figsize=(10, 6))  # Adjust figure size as needed

plt.plot(time, values, label='Series 1', color='b')

# Customize plot elements (optional):
plt.xlabel("Time")
plt.ylabel("Tilt (nrad)")
plt.title("Tilt Signal")
plt.grid(True)
# Adding legend
plt.legend()

plt.show()

In [ ]:
time_column = "time"  # Replace with the actual column name containing time data
value_column = "sensor_value"  # Replace with the actual column name containing values

time = dfs_timeline["time"]
values1 = dfs_timeline["sensor_obs_1"]
values2 = dfs_timeline["sensor_obs_2"]
values3 = dfs_timeline["sensor_obs_3"]
values4 = dfs_timeline["sensor_obs_4"]
values5 = dfs_timeline["sensor_obs_5"]
values6 = dfs_timeline["sensor_obs_6"]
values7 = dfs_timeline["sensor_obs_7"]
values8 = dfs_timeline["sensor_obs_8"]
values9 = dfs_timeline["sensor_obs_9"]
values10 = dfs_timeline["sensor_obs_10"]
values11 = dfs_timeline["sensor_obs_11"]
values12 = dfs_timeline["sensor_obs_12"]

In [ ]:
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plt.plot(time, values1, label='Volcano 1 Observation 1')

# Customize plot elements (optional):
plt.xlabel("Time")
plt.ylabel("Tilt (nrad)")
plt.title("Tilt Signal")
plt.grid(True)
# Adding legend
plt.legend()

plt.show()

In [ ]:
plt.hist(df[value_column], bins=20)  # Adjust bins as needed
plt.xlabel('Tilt Angle (nrad)')
plt.ylabel('Frequency')
plt.title('Distribution of Sensor Value')
plt.show()

In [ ]:
dfs_timeline

In [ ]:
dfs_timeline.iloc[:, 1:].plot(subplots=True, layout=(7, 2), figsize=(20, 14))
plt.show()

In [ ]:
pip install tsfresh


In [ ]:
%%time
from tsfresh import extract_features

In [ ]:
df

In [ ]:
# Generate features for sensor_1 for each segment_id
extracted_features_df = extract_features(timeseries_container=dfs_timeline_seq_with_id,
                                 column_id="id",
                                 column_sort="time",
                                 n_jobs=8,
                                )

In [ ]:
extracted_features_df

#Data Preparation

#Regression Technique Selection

#Model Training

#Model Evaluation

#Results Analysis

#Model Refinement

#Final Evaluation

#Miscellenious